In [1]:
import os
import psycopg 
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [2]:
# подгружаем .env
load_dotenv()

True

In [3]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [4]:
import boto3
import botocore

# Указываем эндпоинт и ключи доступа
s3 = boto3.client(
    's3',
    endpoint_url="https://storage.yandexcloud.net",
    aws_access_key_id=s3_access_key, # Ваш Access Key
    aws_secret_access_key=s3_secret_access_key, 
    config=botocore.client.Config(signature_version='s3v4'),  # Используем правильную конфигурацию
)

# Укажите имя бакета
bucket_name = "s3-student-mle-20250130-582c662a4e"

# Получите список объектов в бакете
try:
    response = s3.list_objects_v2(Bucket=bucket_name)
    
    # Проверьте, есть ли объекты в бакете
    if 'Contents' in response:
        print(f"Файлы в бакете {bucket_name}:")
        for obj in response['Contents']:
            print(f"- {obj['Key']} (размер: {obj['Size']} байт)")
    else:
        print(f"Бакет {bucket_name} пуст.")
except Exception as e:
    print(f"Ошибка при получении списка объектов: {e}")

Файлы в бакете s3-student-mle-20250130-582c662a4e:
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/MLmodel (размер: 803 байт)
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/code/fit.py (размер: 2022 байт)
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/conda.yaml (размер: 415 байт)
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/input_example.json (размер: 382 байт)
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/model.cb (размер: 1070936 байт)
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/python_env.yaml (размер: 115 байт)
- 7/268f18d996e04bef998053cea95bbe22/artifacts/models/requirements.txt (размер: 257 байт)
- 7/fcb9527584ad40d0a202075d7119372d/artifacts/dataframe/columns.txt (размер: 273 байт)
- 7/fcb9527584ad40d0a202075d7119372d/artifacts/dataframe/users_churn.csv (размер: 865601 байт)
- 7/ff7ac48789254ec28f845102ba1379ae/artifacts/models/MLmodel (размер: 803 байт)
- 7/ff7ac48789254ec28f845102ba1379ae/artifacts/models/code/fit.py (размер: 20

In [7]:
import mlflow

# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "test_exp" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
#experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
#with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
#    run_id = run.info.run_id
#    
#    mlflow.log_metric(METRIC_NAME, METRIC_VALUE)

In [8]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20250130_582c662a4e",
    "user": "mle_20250130_582c662a4e",
    "password": "31992be741c54a3f85a88b4082cdcad7",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [9]:
import json
# 1. Название колонок вашего датафрейма запишите в текстовый файл
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(",".join(df.columns))
    
metrics = {
    "num_rows": len(df),
    "num_columns": len(df.columns),
}

with open("metrics.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=4)
    
df.to_csv("dataset.csv", index=False)

In [10]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
    # Посчитать количество уникальных значений в колонке
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

    # Обновить словарь stats
    stats.update(column_stat)

# Общая длина данных
stats["data_length"] = df.shape[0]

# Статистики по monthly_charges
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()

# Статистики по total_charges
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()

# Количество уникальных пользователей
stats["unique_customers_number"] = df["customer_id"].nunique()

# Количество пропусков в end_date
stats["end_date_nan"] = df["end_date"].isna().sum()

In [11]:
import mlflow
import os
import json

# задаём название эксперимента и имя запуска для логирования в MLflow
EXPERIMENT_NAME = "churn_fio_new"
RUN_NAME = "data_check"

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Проверка наличия файлов перед логированием и их создание, если их нет
columns_file_path = "columns.txt"
dataset_file_path = 'users_churn.csv'

# Проверка на существование файла и его создание
if not os.path.exists(columns_file_path):
    with open(columns_file_path, "w", encoding="utf-8") as f:
        f.write(",".join(df.columns))

if not os.path.exists(dataset_file_path):
    df.to_csv(dataset_file_path, index=False)

# Убедимся, что файлы существуют
assert os.path.exists(columns_file_path), f"{columns_file_path} does not exist."
assert os.path.exists(dataset_file_path), f"{dataset_file_path} does not exist."

# Начинаем логирование, передавая experiment_id
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    run_id = run.info.run_id  # Получаем уникальный ID запуска
    
    # Логируем метрики
    mlflow.log_metrics(stats)

    # Логируем файлы как артефакты эксперимента, передавая абсолютный путь
    mlflow.log_artifact('columns.txt', artifact_path="dataframe")
    mlflow.log_artifact('users_churn.csv', artifact_path="dataframe")

# Получаем информацию о запуске
run = mlflow.get_run(run_id)

# Проверяем статус завершённого эксперимента
assert run.info.status == "FINISHED"

# Удаляем файлы после логирования
os.remove(columns_file_path)
os.remove(dataset_file_path)

### Логируем модель

In [16]:
# EXPERIMENT_NAME = "churn_fio_new"

# experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

# if experiment:
#     experiment_id = experiment.experiment_id
#     experiment_name = experiment.name
#     run_count = len(mlflow.search_runs(experiment_ids=[experiment_id]))
    
#     print(f"experiment_id = {experiment_id}")
#     print(f"experiment_name = '{experiment_name}'")
#     print(f"run_count = {run_count}")

Подгружаем нашу модель

In [5]:
# Укажите путь к файлу
import joblib

# Загружаем модель
with open('../churn_model/fitted_model.pkl', 'rb') as fd:
    fit_pipeline = joblib.load(fd)

model = fit_pipeline.named_steps['model']

И теперь подгружаем наши данные для обучения

In [6]:
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

table = 'clean_users_churn'
churn = pd.read_sql(f'select * from {table}', dst_conn)

target = churn['target']

In [7]:
# Получаем наши признаки и скоры (входные и выходные данные)
transformed_data = fit_pipeline.named_steps['preprocessor'].transform(churn)
proba = model.predict_proba(transformed_data)[:, 1]

Логируем 1-ю версию модели

In [15]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

In [ ]:
import mlflow
import numpy as np

# Настройка подключения к tracking и registry
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"

mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')
mlflow.set_registry_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')

EXPERIMENT_NAME = "churn_mmakarov"
RUN_NAME = "m0"
REGISTRY_MODEL_NAME = "churn_model_mmakarov"

# Создаём или получаем эксперимент
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Путь до файла с зависимостями pip
pip_requirements = "../requirements.txt"

# Формирование сигнатуры модели (входы и выходы)
signature = mlflow.models.infer_signature(
    transformed_data,
    proba
)

# Пример входных данных для модели
input_example = [transformed_data[0], transformed_data[1]] 

# Пути до скрипта с обучением модели
code_paths = ["../churn_model/fit.py"]

# Предположим, что модель уже обучена или инициализирована (в данном примере используем модель CatBoost)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    # Логирование модели с передачей всех необходимых параметров и регистрация в реестре
    # model_info = mlflow.pyfunc.log_model(
    #     artifact_path="models",  # Путь для артефактов в реестре
    #     cb_model=model,  # Передаем модель CatBoost с параметром cb_model
    #     pip_requirements=pip_requirements,
    #     signature=signature,
    #     input_example=input_example,
    #     code_paths=code_paths,
    #     registered_model_name=REGISTRY_MODEL_NAME
    # )

    # Логируем метрики (например, точность на обучающих данных)
    roc_auc = roc_auc_score(target, proba)
    precision = precision_score(target, model.predict(transformed_data))
    recall = recall_score(target, model.predict(transformed_data))
    f1 = f1_score(target, model.predict(transformed_data))

    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)


    model_info = mlflow.catboost.log_model(
        cb_model=model,  # Передаем модель CatBoost
        artifact_path="models",  # Путь для артефактов в реестре
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        code_paths=code_paths,
        registered_model_name=REGISTRY_MODEL_NAME
    )

    print(f"Model registered at: {model_info.model_uri}")

Successfully registered model 'churn_model_mmakarov'.
2025/03/15 11:52:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: churn_model_mmakarov, version 1


Model registered at: runs:/e0bf6d32c34d4695b560b2bae1296ed7/models


Created version '1' of model 'churn_model_mmakarov'.


In [10]:
from catboost import CatBoostClassifier

In [16]:
import mlflow
import numpy as np

# Настройка подключения к tracking и registry
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"

mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')
mlflow.set_registry_uri(f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}')

EXPERIMENT_NAME = "churn_mmakarov"
RUN_NAME = "m1"
REGISTRY_MODEL_NAME = "churn_model_mmakarov"

# Создаём или получаем эксперимент
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

model_new = CatBoostClassifier(depth=5, iterations=1001)
model_new.fit(transformed_data, target)

# Путь до файла с зависимостями pip
pip_requirements = "../requirements.txt"

# Формирование сигнатуры модели (входы и выходы)
signature = mlflow.models.infer_signature(
    transformed_data,
    proba
)

# Пример входных данных для модели
input_example = [transformed_data[0], transformed_data[1]] 

# Пути до скрипта с обучением модели
code_paths = ["../churn_model/fit.py"]

# Предположим, что модель уже обучена или инициализирована (в данном примере используем модель CatBoost)

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    # Логирование модели с передачей всех необходимых параметров и регистрация в реестре
    # model_info = mlflow.pyfunc.log_model(
    #     artifact_path="models",  # Путь для артефактов в реестре
    #     cb_model=model,  # Передаем модель CatBoost с параметром cb_model
    #     pip_requirements=pip_requirements,
    #     signature=signature,
    #     input_example=input_example,
    #     code_paths=code_paths,
    #     registered_model_name=REGISTRY_MODEL_NAME
    # )

    # Логируем метрики (например, точность на обучающих данных)
    roc_auc = roc_auc_score(target, model_new.predict_proba(transformed_data)[:, 1])
    precision = precision_score(target, model_new.predict(transformed_data))
    recall = recall_score(target, model_new.predict(transformed_data))
    f1 = f1_score(target, model_new.predict(transformed_data))

    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)


    model_info = mlflow.catboost.log_model(
        cb_model=model,  # Передаем модель CatBoost
        artifact_path="models",  # Путь для артефактов в реестре
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        code_paths=code_paths,
        registered_model_name=REGISTRY_MODEL_NAME
    )

    print(f"Model registered at: {model_info.model_uri}")

Learning rate set to 0.023654
0:	learn: 0.6790975	total: 2.48ms	remaining: 2.48s
1:	learn: 0.6648788	total: 4.94ms	remaining: 2.47s
2:	learn: 0.6516560	total: 8.04ms	remaining: 2.67s
3:	learn: 0.6378380	total: 10.6ms	remaining: 2.64s
4:	learn: 0.6249411	total: 12.9ms	remaining: 2.57s
5:	learn: 0.6137442	total: 15.3ms	remaining: 2.54s
6:	learn: 0.6041292	total: 19.6ms	remaining: 2.78s
7:	learn: 0.5944306	total: 22.3ms	remaining: 2.77s
8:	learn: 0.5849254	total: 25ms	remaining: 2.75s
9:	learn: 0.5761062	total: 27.5ms	remaining: 2.72s
10:	learn: 0.5684946	total: 29.9ms	remaining: 2.69s
11:	learn: 0.5610197	total: 32.2ms	remaining: 2.65s
12:	learn: 0.5526794	total: 34.5ms	remaining: 2.62s
13:	learn: 0.5470833	total: 36.9ms	remaining: 2.6s
14:	learn: 0.5407747	total: 39.6ms	remaining: 2.6s
15:	learn: 0.5338307	total: 42.9ms	remaining: 2.64s
16:	learn: 0.5297786	total: 44.7ms	remaining: 2.59s
17:	learn: 0.5239302	total: 47.6ms	remaining: 2.6s
18:	learn: 0.5190328	total: 50.1ms	remaining: 2.5

Registered model 'churn_model_mmakarov' already exists. Creating a new version of this model...
2025/03/15 12:09:28 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: churn_model_mmakarov, version 2


Model registered at: runs:/627281a1ee9b4c8d86f3e30d0a3d16cd/models


Created version '2' of model 'churn_model_mmakarov'.


In [ ]:
import mlflow
import os

EXPERIMENT_NAME = "churn_mmakarov"
RUN_NAME = "m1"
REGISTRY_MODEL_NAME = "churn_model_mmakarov"


TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('S3_ACCESS_KEY')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('S3_SECRET_KEY')

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_experiment("churn_mmakarov")

client = mlflow.MlflowClient() 

models = client.search_model_versions(
    filter_string=f"name = '{REGISTRY_MODEL_NAME}'"
)

print(f"Model info:\n {models}")

model_name_1 = models[-1].name
model_version_1 = models[-1].version
model_stage_1 = models[-1].current_stage

model_name_2 = models[-2].name
model_version_2 = models[-2].version
model_stage_2 = models[-2].current_stage


print(f"Текущий stage модели 1: {model_stage_1}")
print(f"Текущий stage модели 2: {model_stage_2}")

# Поменяйте статус каждой модели

client.transition_model_version_stage(model_name_1, model_version_1, 'production')
client.transition_model_version_stage(model_name_2, model_version_2, 'staging')
# Переименуйте модель в реестре

client.rename_registered_model(name=REGISTRY_MODEL_NAME, new_name=f'{REGISTRY_MODEL_NAME}_b2c')